# Setup

**Download deepdrive_course repository when running in Google Colab (to have access to libraries)**

In [ ]:
import sys

in_colab = "google.colab" in sys.modules

if in_colab:
    !git clone https://github.com/abojda/deepdrive_course.git dd_course
    !pip install dd_course/ -q

In [2]:
!pip install neural-compressor -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00


In [3]:
import torch
import pytorch_lightning as pl

# Model

## Download model weigths

In [4]:
from deepdrive_course.utils import download_from_mega_nz

# resnet50-tl-ft_onecycle_lr0.0005-drop_0.3-basic_aug---val_loss=0.15.ckpt
checkpoint = download_from_mega_nz(
    "https://mega.nz/file/1mkigBqC#N3wyKt1N15DydkYNoniQvkcLqYCbPMDohvHY7UsVAGw"
)

## Load model

In [ ]:
from deepdrive_course.resisc45.modules import ResiscLit

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResiscLit.load_from_checkpoint(checkpoint, map_location=device)

# Load dataset and dataloader

In [ ]:
from deepdrive_course.utils import stratified_train_test_split
from deepdrive_course.resisc45.datasets import RESISC45Albumentations
from deepdrive_course.resisc45.transforms import get_transform

train_transform = get_transform("albumentations_imagenet_norm")
test_transform = get_transform("albumentations_imagenet_norm")

full_ds = RESISC45Albumentations(root="data,", download=True)

train_ds, test_ds, _, _ = stratified_train_test_split(
    dataset=full_ds,
    train_size=0.8,
    train_transform=train_transform,
    test_transform=test_transform,
)

In [7]:
from neural_compressor.data.dataloaders.pytorch_dataloader import PyTorchDataLoader

batch_size = 100

train_dl_nc = PyTorchDataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
)

test_dl_nc = PyTorchDataLoader(
    test_ds,
    batch_size=batch_size,
    shuffle=False,
)

# Quantization

In [8]:
from neural_compressor.config import (
    PostTrainingQuantConfig,
    TuningCriterion,
    AccuracyCriterion,
)

accuracy_criterion = AccuracyCriterion(tolerable_loss=0.01)
tuning_criterion = TuningCriterion(max_trials=10)

conf = PostTrainingQuantConfig(
    approach="static",
    backend="default",
    tuning_criterion=tuning_criterion,
    accuracy_criterion=accuracy_criterion,
)

In [9]:
from neural_compressor.quantization import fit
from neural_compressor import Metric

accuracy_metric = Metric("Accuracy")

q_model = fit(
    model=model.model,
    conf=conf,
    calib_dataloader=train_dl_nc,
    eval_dataloader=test_dl_nc,
    eval_metric=accuracy_metric,
)

2023-07-04 13:00:26 [INFO] Start auto tuning.
2023-07-04 13:00:26 [INFO] Create evaluation function according to evaluation dataloader and metric                and Execute the tuning process.
2023-07-04 13:00:26 [INFO] Adaptor has 4 recipes.
2023-07-04 13:00:26 [INFO] 0 recipes specified by user.
2023-07-04 13:00:26 [INFO] 3 recipes require future tuning.
2023-07-04 13:00:26 [INFO] *** Initialize auto tuning
2023-07-04 13:00:26 [INFO] {
2023-07-04 13:00:26 [INFO]     'PostTrainingQuantConfig': {
2023-07-04 13:00:26 [INFO]         'AccuracyCriterion': {
2023-07-04 13:00:26 [INFO]             'criterion': 'relative',
2023-07-04 13:00:26 [INFO]             'higher_is_better': True,
2023-07-04 13:00:26 [INFO]             'tolerable_loss': 0.01,
2023-07-04 13:00:26 [INFO]             'absolute': None,
2023-07-04 13:00:26 [INFO]             'keys': <bound method AccuracyCriterion.keys of <neural_compressor.config.AccuracyCriterion object at 0x7f2213412ec0>>,
2023-07-04 13:00:26 [INFO]      

# Save models

In [ ]:
q_model.save("q_model")
torch.save(model.model, "regular_model.pt")

In [25]:
!ls -sh regular_model.pt
!ls -sh q_model/best_model.pt

91M regular_model.pt
24M q_model/best_model.pt
